In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('olist').getOrCreate()

25/07/26 03:45:55 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
hdfs_path = '/data/olist/'
customers_df = spark.read.csv(hdfs_path+'olist_customers_dataset.csv',header=True, inferSchema=True)
products_df = spark.read.csv(hdfs_path+'olist_products_dataset.csv',header=True,inferSchema=True)
order_df = spark.read.csv(hdfs_path+'olist_orders_dataset.csv',header = True , inferSchema = True)
item_df = spark.read.csv(hdfs_path+'olist_order_items_dataset.csv',header = True, inferSchema=True)
payment_df = spark.read.csv(hdfs_path+'olist_order_payments_dataset.csv',header = True , inferSchema = True)
reviews_df = spark.read.csv(hdfs_path+'olist_order_reviews_dataset.csv',header = True , inferSchema = True)
location_df = spark.read.csv(hdfs_path+'olist_geolocation_dataset.csv',header = True , inferSchema = True)
seller_df = spark.read.csv(hdfs_path+'olist_sellers_dataset.csv',header = True , inferSchema = True)
trnaslation_df = spark.read.csv(hdfs_path+'product_category_name_translation.csv',header = True , inferSchema = True)

In [3]:
# caching the most used dfs
order_df.cache()
customers_df.cache()
item_df.cache()

DataFrame[order_id: string, order_item_id: int, product_id: string, seller_id: string, shipping_limit_date: timestamp, price: double, freight_value: double]

In [4]:
# all joins
order_item_joined_df = order_df.join(item_df,'order_id','inner')
order_items_products = order_item_joined_df.join(products_df,'product_id','inner')
order_item_product_seller = order_items_products.join(seller_df,'seller_id','inner')
full_orders_df = order_item_product_seller.join(customers_df,'customer_id','inner')

In [5]:
from pyspark.sql.functions import *

full_orders_df =  full_orders_df.join(broadcast(location_df)\
                                      ,full_orders_df.customer_zip_code_prefix == location_df.geolocation_zip_code_prefix,'left')

In [6]:
full_orders_df = full_orders_df.join(broadcast(reviews_df),'order_id','left')
full_orders_df = full_orders_df.join(payment_df,'order_id','left')
full_orders_df.cache()

25/07/26 03:46:24 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[order_id: string, customer_id: string, seller_id: string, product_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp, order_item_id: int, shipping_limit_date: timestamp, price: double, freight_value: double, product_category_name: string, product_name_lenght: int, product_description_lenght: int, product_photos_qty: int, product_weight_g: int, product_length_cm: int, product_height_cm: int, product_width_cm: int, seller_zip_code_prefix: int, seller_city: string, seller_state: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: string, customer_state: string, geolocation_zip_code_prefix: int, geolocation_lat: double, geolocation_lng: double, geolocation_city: string, geolocation_state: string, review_id: string, review_score: string, review_comment_title: string, review_commen

In [7]:
full_orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [8]:
full_orders_df.show(5)

+--------------------+--------------------+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+-------------------+-----+-------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+----------------------+-------------+------------+--------------------+------------------------+--------------------+--------------+---------------------------+-------------------+------------------+--------------------+-----------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+------------------+------------+--------------------+-------------+
|            order_id|         customer_id|           seller_id|          product_id|order_status|order_purchase_timestamp|  or

In [9]:
# Total Revenue and Average Order Value (AOV) per customer
customer_spending_df = full_orders_df.groupBy('customer_id')\
.agg(
    count('order_id').alias('total_orders'),
    sum('price').alias('total_spent'),
    round(avg('price'),2).alias('AOV') # round of average number with 2 decimanl point precision
).orderBy(desc('total_spent'))

customer_spending_df.show()

+--------------------+------------+------------------+-------+
|         customer_id|total_orders|       total_spent|    AOV|
+--------------------+------------+------------------+-------+
|d3e82ccec3cb5f956...|        6876|         6662844.0|  969.0|
|df55c14d1476a9a34...|         743|         3565657.0| 4799.0|
|fe5113a38e3575c04...|        2292|         3293604.0| 1437.0|
|ec5b2ba62e5743423...|        1428|         2556120.0| 1790.0|
|63b964e79dee32a35...|        6072|         2501664.0|  412.0|
|46bb3c0b1a65c8399...|         748|         2336752.0| 3124.0|
|05455dfa7cd02f13d...|        2184| 2160194.400000087|  989.1|
|3690e975641f01bd0...|         802|         2124498.0| 2649.0|
|349509b216bd5ec11...|         743|         1923627.0| 2589.0|
|695476b5848d64ba0...|         687|1820543.1299999943|2649.99|
|73236a0796f53d60d...|         832|         1755520.0| 2110.0|
|cc803a2c412833101...|         762|         1676400.0| 2200.0|
|1ff773612ab8934db...|        5820|1658641.7999999512| 

In [10]:
full_orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [11]:
# Seller performance matrix
seller_performance_df = full_orders_df.groupBy('seller_id')\
.agg(
    count('order_id').alias('total_orders'),
    sum('price').alias('Total_revenue'),
    round(avg('review_score'),2).alias('average_review'),
    round(stddev('price'),2).alias('price_variability') # calculating standard deviation for price
).orderBy(desc('Total_revenue'))

seller_performance_df.show()

+--------------------+------------+--------------------+--------------+-----------------+
|           seller_id|total_orders|       Total_revenue|average_review|price_variability|
+--------------------+------------+--------------------+--------------+-----------------+
|4869f7a5dfa277a7d...|      184587| 3.613871731999314E7|          4.09|           111.65|
|53243585a1d6dc264...|       54514|3.4291592950000696E7|          4.12|           499.65|
|4a3ca9315b744ce9f...|      330661| 3.375957084001202E7|          3.77|            59.37|
|7c67e1448b00f6e96...|      233306|3.2282321790021457E7|          3.42|            50.39|
|fa1c13f2614d7b5c4...|       87686|3.0139386310006626E7|          4.38|            307.7|
|da8622b14eb17ae28...|      264433| 2.985766973003611E7|          3.98|            72.92|
|7e93a43ef30c4f03f...|       50226| 2.631570630000355E7|          4.15|           377.24|
|1025f0e2d44d7041d...|      229587|2.2937518520012792E7|          3.89|             84.3|
|46dc3b2cc

In [12]:
# Product popularity
product_matrics_df = full_orders_df.groupBy('product_id')\
.agg(
    count('order_id').alias('Total_sales'),
    sum('price').alias('Total_revenue'),
    round(avg('price'),2).alias('Average_price'),
    round(stddev('price'),2).alias('Price_deviation'),
    collect_list('seller_id').alias('Seller_list'), # collect_;ist will collect all sellers and make a list out of it .
    collect_set('seller_id').alias('Unique_sellers') # collect_set will collect the unique sellers  
).orderBy(desc('Total_sales'))

product_matrics_df.show()

+--------------------+-----------+------------------+-------------+---------------+--------------------+--------------------+
|          product_id|Total_sales|     Total_revenue|Average_price|Price_deviation|         Seller_list|      Unique_sellers|
+--------------------+-----------+------------------+-------------+---------------+--------------------+--------------------+
|aca2eb7d00ea1a7b8...|      86740| 6164630.299996213|        71.07|           3.17|[955fee9216a65b61...|[955fee9216a65b61...|
|422879e10f4668299...|      81110| 4442791.509997526|        54.77|           4.46|[1f50f920176fa81d...|[1f50f920176fa81d...|
|99a4788cb24856965...|      78775| 6921762.709996367|        87.87|           4.08|[4a3ca9315b744ce9...|[4a3ca9315b744ce9...|
|389d119b48cf3043d...|      60248|3280533.1299988106|        54.45|           4.37|[1f50f920176fa81d...|[1f50f920176fa81d...|
|d1c427060a0f73f6b...|      59274| 8220103.330002412|       138.68|          16.58|[a1043bafd471dff5...|[a1043bafd471d

In [13]:
full_orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [14]:
# Customer retention Analysis (First and last order)
customer_retention_df = full_orders_df.groupBy('customer_id')\
.agg(
    first('order_purchase_timestamp').alias('first_order_date'),
    last('order_purchase_timestamp').alias('last_order_date'),
    count('order_id').alias('Total_orders'),
    round(avg('price'),2).alias('Average_price')
).orderBy(desc('Total_orders'))

customer_retention_df.show()

+--------------------+-------------------+-------------------+------------+-------------+
|         customer_id|   first_order_date|    last_order_date|Total_orders|Average_price|
+--------------------+-------------------+-------------------+------------+-------------+
|351e40989da90e704...|2017-07-13 10:42:37|2017-07-13 10:42:37|       11427|        85.99|
|50920f8cd0681fd86...|2018-01-27 11:28:32|2018-01-27 11:28:32|       10752|        43.82|
|9b43e2a62de9bab3a...|2017-05-25 22:27:50|2017-05-25 22:27:50|        8556|         26.4|
|270c23a11d024a44c...|2017-08-08 20:26:31|2017-08-08 20:26:31|        8001|        36.59|
|5c87184371002d49e...|2018-01-05 19:15:37|2018-01-05 19:15:37|        6876|        12.49|
|d3e82ccec3cb5f956...|2017-03-18 14:28:34|2017-03-18 14:28:34|        6876|        969.0|
|d5f2b3f597c7ccafb...|2017-12-13 14:21:15|2017-12-13 14:21:15|        6706|         59.0|
|c2f18647725395af4...|2018-03-06 19:21:47|2018-03-06 19:21:47|        6612|         34.9|
|24e7dc2ff

## Orders could be counted twice or multiple times as on multiple products we can have same order_id , so instead of count we can use , countDistinct and for fist and last we could use min max functions instead of first and last ones

In [15]:
customer_retention_refined_df = full_orders_df.groupBy('customer_id')\
.agg(
    max('order_purchase_timestamp').alias('first_order_date'),
    min('order_purchase_timestamp').alias('last_order_date'),
    count('order_id').alias('Total_orders'),
    round(avg('price'),2).alias('Average_price')
).orderBy(desc('Total_orders'))

customer_retention_refined_df.show()

+--------------------+-------------------+-------------------+------------+-------------+
|         customer_id|   first_order_date|    last_order_date|Total_orders|Average_price|
+--------------------+-------------------+-------------------+------------+-------------+
|351e40989da90e704...|2017-07-13 10:42:37|2017-07-13 10:42:37|       11427|        85.99|
|50920f8cd0681fd86...|2018-01-27 11:28:32|2018-01-27 11:28:32|       10752|        43.82|
|9b43e2a62de9bab3a...|2017-05-25 22:27:50|2017-05-25 22:27:50|        8556|         26.4|
|270c23a11d024a44c...|2017-08-08 20:26:31|2017-08-08 20:26:31|        8001|        36.59|
|5c87184371002d49e...|2018-01-05 19:15:37|2018-01-05 19:15:37|        6876|        12.49|
|d3e82ccec3cb5f956...|2017-03-18 14:28:34|2017-03-18 14:28:34|        6876|        969.0|
|d5f2b3f597c7ccafb...|2017-12-13 14:21:15|2017-12-13 14:21:15|        6706|         59.0|
|c2f18647725395af4...|2018-03-06 19:21:47|2018-03-06 19:21:47|        6612|         34.9|
|24e7dc2ff

In [16]:
# Monthly revenue per seller
monthly_revenue_seller_df = full_orders_df.withColumn('order_year', year('order_purchase_timestamp'))\
                                          .withColumn('order_month',month('order_purchase_timestamp'))\
                                          .groupBy('seller_id','order_year','order_month').agg(sum('price').alias('Total_revenue')).orderBy(desc('Total_revenue'))

monthly_revenue_seller_df.show()

+--------------------+----------+-----------+------------------+
|           seller_id|order_year|order_month|     Total_revenue|
+--------------------+----------+-----------+------------------+
|53243585a1d6dc264...|      2017|         10| 8660974.320000233|
|53243585a1d6dc264...|      2017|          9| 8121993.890000252|
|ccb83a794700270fd...|      2017|          3|         6662844.0|
|53243585a1d6dc264...|      2017|          8|         6430396.0|
|59417c56835dd8e2e...|      2017|          4|         6245429.0|
|7d13fca1522535862...|      2018|          5| 4505180.299999628|
|4869f7a5dfa277a7d...|      2018|          3| 4429672.499999713|
|7e93a43ef30c4f03f...|      2017|          5| 4356431.119999937|
|b839e41795b7f3ad9...|      2018|          6|         4205530.0|
|7d13fca1522535862...|      2018|          4|  4200775.77999977|
|46dc3b2cc0980fb8e...|      2017|         11|4048986.0300003635|
|955fee9216a65b617...|      2018|          5|4047132.5399993127|
|46dc3b2cc0980fb8e...|   

### Extended Enrichment in data

In [17]:
full_orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [18]:
full_orders_df.select('order_status').show()

+------------+
|order_status|
+------------+
|   delivered|
|   delivered|
|   delivered|
|   delivered|
|   delivered|
|   delivered|
|   delivered|
|   delivered|
|   delivered|
|   delivered|
|   delivered|
|   delivered|
|   delivered|
|   delivered|
|   delivered|
|   delivered|
|   delivered|
|   delivered|
|   delivered|
|   delivered|
+------------+
only showing top 20 rows



We want to biffurcate the delivered and ongoing couries differently.

In [19]:
full_orders_df.select('order_status').distinct().show()

+------------+
|order_status|
+------------+
|   delivered|
|    canceled|
|     shipped|
|    approved|
|  processing|
|    invoiced|
| unavailable|
+------------+



In [20]:
# Creating new column for delivery status as delivered would be 1  and otherwise is 0
full_orders_df = full_orders_df.withColumn('is_delivered',when(col('order_status') == 'delivered',lit(1)).otherwise(lit(0)))\
.withColumn('is_cancelled',when(col('order_status') == 'canceled' , lit(1)).otherwise(lit(0)))

In [21]:
full_orders_df.select('order_status','is_delivered','is_cancelled').show()

+------------+------------+------------+
|order_status|is_delivered|is_cancelled|
+------------+------------+------------+
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
+------------+------------+------------+
only showing top

In [22]:
full_orders_df.where(full_orders_df['order_status'] == 'canceled').select('order_status','is_delivered','is_cancelled').show()

+------------+------------+------------+
|order_status|is_delivered|is_cancelled|
+------------+------------+------------+
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
|    canceled|           0|           1|
+------------+------------+------------+
only showing top

In [23]:
full_orders_df.where(full_orders_df['order_status'] == 'delivered').select('order_status','is_delivered','is_cancelled').show()

+------------+------------+------------+
|order_status|is_delivered|is_cancelled|
+------------+------------+------------+
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
|   delivered|           1|           0|
+------------+------------+------------+
only showing top

In [24]:
full_orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [25]:
# Order Revenue calculation
full_orders_df = full_orders_df.withColumn('order_revenue',col('price')+col('freight_value'))

In [26]:
full_orders_df.select('price','freight_value','order_revenue').show()

+-----+-------------+-------------+
|price|freight_value|order_revenue|
+-----+-------------+-------------+
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
| 58.9|        13.29|        72.19|
+-----+-------------+-------------+
only showing top 20 rows



In [27]:
customer_spending_df.show()

+--------------------+------------+------------------+-------+
|         customer_id|total_orders|       total_spent|    AOV|
+--------------------+------------+------------------+-------+
|d3e82ccec3cb5f956...|        6876|         6662844.0|  969.0|
|df55c14d1476a9a34...|         743|         3565657.0| 4799.0|
|fe5113a38e3575c04...|        2292|         3293604.0| 1437.0|
|ec5b2ba62e5743423...|        1428|         2556120.0| 1790.0|
|63b964e79dee32a35...|        6072|         2501664.0|  412.0|
|46bb3c0b1a65c8399...|         748|         2336752.0| 3124.0|
|05455dfa7cd02f13d...|        2184| 2160194.400000087|  989.1|
|3690e975641f01bd0...|         802|         2124498.0| 2649.0|
|349509b216bd5ec11...|         743|         1923627.0| 2589.0|
|695476b5848d64ba0...|         687|1820543.1299999943|2649.99|
|73236a0796f53d60d...|         832|         1755520.0| 2110.0|
|cc803a2c412833101...|         762|         1676400.0| 2200.0|
|1ff773612ab8934db...|        5820|1658641.7999999512| 

In [31]:
#Customer Segmentation based on spending
customer_spending_df = customer_spending_df.withColumn('Customer_segment'\
                                                       ,when(col('AOV') >= 1200 ,'High_Value').when( (col('AOV')<1200) & (col('AOV') >=500) ,'Medium_value').otherwise('Low_value'))

In [32]:
customer_spending_df.show()

+--------------------+------------+------------------+-------+----------------+
|         customer_id|total_orders|       total_spent|    AOV|Customer_segment|
+--------------------+------------+------------------+-------+----------------+
|d3e82ccec3cb5f956...|        6876|         6662844.0|  969.0|    Medium_value|
|df55c14d1476a9a34...|         743|         3565657.0| 4799.0|      High_Value|
|fe5113a38e3575c04...|        2292|         3293604.0| 1437.0|      High_Value|
|ec5b2ba62e5743423...|        1428|         2556120.0| 1790.0|      High_Value|
|63b964e79dee32a35...|        6072|         2501664.0|  412.0|       Low_value|
|46bb3c0b1a65c8399...|         748|         2336752.0| 3124.0|      High_Value|
|05455dfa7cd02f13d...|        2184| 2160194.400000087|  989.1|    Medium_value|
|3690e975641f01bd0...|         802|         2124498.0| 2649.0|      High_Value|
|349509b216bd5ec11...|         743|         1923627.0| 2589.0|      High_Value|
|695476b5848d64ba0...|         687|18205

In [33]:
full_orders_df = full_orders_df.join(customer_spending_df.select('customer_id','Customer_segment'),'customer_id',how='left')

In [34]:
full_orders_df.select('customer_id','Customer_segment').show()

+--------------------+----------------+
|         customer_id|Customer_segment|
+--------------------+----------------+
|c08ef557085ca9fb0...|       Low_value|
|c08ef557085ca9fb0...|       Low_value|
|c08ef557085ca9fb0...|       Low_value|
|c08ef557085ca9fb0...|       Low_value|
|c08ef557085ca9fb0...|       Low_value|
|c08ef557085ca9fb0...|       Low_value|
|c08ef557085ca9fb0...|       Low_value|
|c08ef557085ca9fb0...|       Low_value|
|c08ef557085ca9fb0...|       Low_value|
|c08ef557085ca9fb0...|       Low_value|
|c08ef557085ca9fb0...|       Low_value|
|c08ef557085ca9fb0...|       Low_value|
|c08ef557085ca9fb0...|       Low_value|
|c08ef557085ca9fb0...|       Low_value|
|c08ef557085ca9fb0...|       Low_value|
|c08ef557085ca9fb0...|       Low_value|
|c08ef557085ca9fb0...|       Low_value|
|c08ef557085ca9fb0...|       Low_value|
|c08ef557085ca9fb0...|       Low_value|
|c08ef557085ca9fb0...|       Low_value|
+--------------------+----------------+
only showing top 20 rows



In [35]:
full_orders_df.select('order_purchase_timestamp').show(5)

+------------------------+
|order_purchase_timestamp|
+------------------------+
|     2017-09-13 08:59:02|
|     2017-09-13 08:59:02|
|     2017-09-13 08:59:02|
|     2017-09-13 08:59:02|
|     2017-09-13 08:59:02|
+------------------------+
only showing top 5 rows



In [36]:
# Hourly Order Distribution
full_orders_df =  full_orders_df.withColumn('hour_of_day',expr('hour(order_purchase_timestamp)'))

In [37]:
full_orders_df.select('order_purchase_timestamp','hour_of_day').show(5)

+------------------------+-----------+
|order_purchase_timestamp|hour_of_day|
+------------------------+-----------+
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
|     2017-09-13 08:59:02|          8|
+------------------------+-----------+
only showing top 5 rows



In [40]:
# Weekday vs weekend order , dayofweek function in pyspark - 1-Sunday , 7-Saturday 
full_orders_df = full_orders_df.withColumn('order_day_type'\
                                           ,when(dayofweek('order_purchase_timestamp').isin(1,7),lit('weekend')).otherwise(lit('weekday')))

In [41]:
full_orders_df.select('order_purchase_timestamp','order_day_type').show()

+------------------------+--------------+
|order_purchase_timestamp|order_day_type|
+------------------------+--------------+
|     2017-09-13 08:59:02|       weekday|
|     2017-09-13 08:59:02|       weekday|
|     2017-09-13 08:59:02|       weekday|
|     2017-09-13 08:59:02|       weekday|
|     2017-09-13 08:59:02|       weekday|
|     2017-09-13 08:59:02|       weekday|
|     2017-09-13 08:59:02|       weekday|
|     2017-09-13 08:59:02|       weekday|
|     2017-09-13 08:59:02|       weekday|
|     2017-09-13 08:59:02|       weekday|
|     2017-09-13 08:59:02|       weekday|
|     2017-09-13 08:59:02|       weekday|
|     2017-09-13 08:59:02|       weekday|
|     2017-09-13 08:59:02|       weekday|
|     2017-09-13 08:59:02|       weekday|
|     2017-09-13 08:59:02|       weekday|
|     2017-09-13 08:59:02|       weekday|
|     2017-09-13 08:59:02|       weekday|
|     2017-09-13 08:59:02|       weekday|
|     2017-09-13 08:59:02|       weekday|
+------------------------+--------

In [42]:
full_orders_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [43]:
spark.stop()